In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
%autoreload 2
import sys
sys.path.insert(0, '../')
from utils import *

In [11]:
coef = 'nest1.blca_mutsig_diff_nneg_log_ps5.glmnet-ms-impact.coef'
mat = 'nest1_conn.txt'
signal = 'build_20181101_blca_diff_nneg_log_ps5.txt'

In [41]:
%time outputs = redistribute_gene_score(coef, mat, signal)

CPU times: user 3.62 s, sys: 154 ms, total: 3.77 s
Wall time: 3.78 s
